In [1]:
from data_security.securables import *

# https://docs.azuredatabricks.net/user-guide/secrets/index.html#secrets-user-guide
# encryptionKey = dbutils.preview.secret.get(scope = "bk", key = "encryption_key")

# BAD PRACTICE - do not store sensitive data like this in plain text in a notebook
# key length 32
encryptionKey = str('28cf740e10ea6502422755ec494fda63')

print(func_encrypt(encryptionKey,'Test Data 123'))
print(func_decrypt(encryptionKey,func_encrypt(encryptionKey,'Test Data 123')))

CVNzSYfZMrojcPM679doOc8vkLzvBOxJWIHFI5b20fhY15zt9/lzaog94EM6fbWmbHb5xFwZe22PgZ6lFGLqUA==
Test Data 123

In [2]:
from pyspark.sql.types import *
schema = StructType([StructField("age", IntegerType()),
                     StructField("workclass", StringType()),
                     StructField("education", StringType()),
                     StructField("income", StringType())])
list = [
  [50, "Self-emp-not-inc","Bachelors","<=50K"],
  [28, "Private","HS-grad",">50K"],
  [27, "Public","Masters",">50K"]
]

df = spark.createDataFrame(list,schema=schema)

In [3]:
from pyspark.sql.functions import udf, lit, md5
from pyspark.sql.types import StringType
from data_security.securables import *

encrypt_udf = udf(func_encrypt, StringType())
decrypt_udf = udf(func_decrypt, StringType())

encrypted = df.select("education", "income" , encrypt_udf(lit(encryptionKey),"income").alias("encrypted_income"))
display(encrypted)

education,income,encrypted_income
Bachelors,<=50K,AjD2Gz2alIWU3kspbuFPZnogpTDrDwj1l2CbxcY5sFtrqmFR9RsClGTDe6I/COVTCYgKugfDEQod4b9pvHbGzQ==
HS-grad,>50K,J9BoRzCvR9R5ikm8ztUgvGSIx6YzXstydqj4hNi0Lo0pUcsYdqQW6uiMNrpUM8h/6a64sTY5crZLwEtFY+GhKQ==
Masters,>50K,XClQDG5MsABRJVmaPMvEDiUGWcTdQ//melc6iW0ohBkVFrndJ2nvID42EFTcbNLEDHtEGyipbbRFuRL1U+nYtA==


In [4]:
display(encrypted.select("education","encrypted_income",decrypt_udf(lit(encryptionKey),"encrypted_income").alias("decrypted_income")))

education,encrypted_income,decrypted_income
Bachelors,hEI63HjN41R7Zakr4YV36Jq4iVhgHnPr6+IjX0rXWhCcqezLeXufQocyhEg+/ptuMVcgGWlUOi9dUqcsWdhHUQ==,<=50K
HS-grad,TRMKEMjVv+wso9B3F+nyMD6rrt+qc4GWeZMqGjY2Dvswg4/KMfK5TdfEhk0RJQpOnmMXpEiOY1NJfqaywMM+yQ==,>50K
Masters,kbSL9b1WAwxMxJQWrkeSCRPFh6ZoZQvxQerMNScSo7pGiLdHAY9BX+aFhWiTm2BZHdrMNW4jyAeDt5of6EUKtg==,>50K
